<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5bf690fc46970c50e69a20a47df70461184fce02f4c68a9f20c406e406a46d32
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 13:44:44
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: nan (nan%)
Current PnL: -17.28 L (-11.62%)
CY Booked + Current PnL: -6.25 L (-4.21%)
-------------------
Total profit:  2.20 L
Total loss:  -19.48 L
-------------------
Total Booked + Current PnL: 20.73 L (17.03%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.77%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.18 L (58.61%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,65.0,M-LC,4.24,219672.0,18192.0,9029.0,0.51,9.03,4.11,13.51,60.0,2.01,1.57,22.76,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.75,57.0,H-LC,11.05,172381.0,16204.0,9895.0,0.24,10.38,5.74,16.71,22.0,1.64,1.23,29.62,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,103.20,56.0,M-SC,4.44,87541.0,-13236.0,13332.0,-0.58,-13.13,15.23,0.09,245.0,-0.99,0.63,15.01,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114756.0,14252.0,16387.0,-0.12,14.18,14.28,30.49,79.0,0.87,0.82,51.18,MH,ATH,METALS
42,ITC,452.00,-37.50,53.0,H-LC,2.87,201884.0,1746.0,19139.0,0.05,0.87,9.48,10.44,4.0,0.09,1.44,6.48,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-30.16,53.0,H-SC,17.34,199560.0,-394.0,97625.0,-0.10,-0.20,48.92,48.63,115.0,-0.00,1.43,11.33,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.65,44.0,H-MC,2.22,136470.0,682.0,42074.0,-0.22,0.50,30.83,31.48,107.0,0.02,0.98,12.68,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,59.0,H-MC,5.80,180300.0,492.0,111912.0,-0.64,0.27,62.07,62.52,88.0,0.00,1.29,34.45,XR,NTT,BANKS
52,PGHH,17973.08,-31.08,50.0,H-MC,4.39,199425.0,-1395.0,70178.0,0.35,-0.69,35.19,34.25,80.0,-0.02,1.43,4.47,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296383.0,-19459.0,132128.0,-1.51,-6.16,44.58,35.67,5.0,-0.15,2.12,6.84,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.45,44.0,H-LC,8.15,218381.0,-33387.0,86588.0,-0.84,-13.26,39.65,21.13,27.0,-0.39,1.56,17.47,X40,ATH,PAINTS
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.02,230837.0,-11087.0,74353.0,0.26,-4.58,32.21,26.15,8.0,-0.15,1.65,9.08,X40,ATH,IT
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.81,213254.0,8754.0,32671.0,0.54,4.28,15.32,20.25,10.0,0.27,1.53,13.39,X40N,BTT,PHARMA
81,UNITDSPR,1644.00,-13.09,51.0,H-LC,7.88,227728.0,-6416.0,55042.0,0.66,-2.74,24.17,20.77,86.0,-0.12,1.63,3.99,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.60,62.0,H-LC,1.01,241538.0,-4613.0,83814.0,0.49,-1.87,34.70,32.18,16.0,-0.06,1.73,33.49,X200,ATH,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.87,201884.0,1746.0,19139.0,0.05,0.87,9.48,10.44,4.0,0.09,1.44,6.48,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.22,58.0,H-LC,3.78,282532.0,17106.0,39696.0,-0.42,6.44,14.05,21.40,11.0,0.43,2.02,13.52,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.91,55.0,H-LC,5.58,218400.0,8034.0,20748.0,0.05,3.82,9.50,13.68,37.0,0.39,1.56,20.97,XY25,NTT,REFINERIES
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296383.0,-19459.0,132128.0,-1.51,-6.16,44.58,35.67,5.0,-0.15,2.12,6.84,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,61.98,69.0,H-SC,8.57,140588.0,-42187.0,153255.0,4.74,-23.08,109.01,60.77,125.0,-0.28,1.01,25.92,XR,NTT,JEWELLERY
55,RAJOOENG,143.33,-33.76,46.0,H-SC,10.61,100540.0,-1960.0,42790.0,3.44,-1.91,42.56,39.83,114.0,-0.05,0.72,5.09,AR,ATH,MISC
22,COFFEEDAY,80.00,-41.65,56.0,L-SC,41.96,87610.0,-25939.0,65988.0,2.72,-22.84,75.32,35.27,268.0,-0.39,0.63,114.43,XR,NTT,HOTELS
77,TRIDENT,48.00,-5.57,63.0,M-SC,5.37,73433.0,-18822.0,45125.0,2.52,-20.40,61.45,28.51,224.0,-0.42,0.53,25.66,XR,NTT,TEXTILES
10,BAJAJHFL,152.00,-9.82,61.0,H-MC,6.07,189846.0,-10660.0,59441.0,2.13,-5.32,31.31,24.33,90.0,-0.18,1.36,9.48,X40N,BTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SHALBY,327.00,1065.87,65.0,M-SC,2.56,158461.0,-22682.0,68154.0,-1.75,-12.52,43.01,25.10,235.0,-0.33,1.13,26.99,XY24,NTT,HEALTHCARE
76,TITAGARH,1548.00,-0.35,65.0,H-SC,4.17,163362.0,-28653.0,107541.0,-1.56,-14.92,65.83,41.08,158.0,-0.27,1.17,38.43,XY24,NTT,ENGINEERING
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296383.0,-19459.0,132128.0,-1.51,-6.16,44.58,35.67,5.0,-0.15,2.12,6.84,X40N,ATH,FMCG
4,ANGELONE,3033.00,3.06,35.0,H-SC,9.28,182302.0,-8704.0,66394.0,-1.28,-4.56,36.42,30.21,157.0,-0.13,1.30,15.37,X40N,NTT,FINANCE
79,UJJIVANSFB,60.00,97.20,57.0,H-SC,12.43,124551.0,-17928.0,37452.0,-1.26,-12.58,30.07,13.70,163.0,-0.48,0.89,47.00,OX40N,NTT,BANKS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,97.20,57.0,H-SC,12.43,124551.0,-17928.0,37452.0,-1.26,-12.58,30.07,13.70,163.0,-0.48,0.89,47.00,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,103.20,56.0,M-SC,4.44,87541.0,-13236.0,13332.0,-0.58,-13.13,15.23,0.09,245.0,-0.99,0.63,15.01,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.29,45.0,H-SC,2.16,145532.0,-30371.0,72388.0,0.13,-17.27,49.74,23.89,149.0,-0.42,1.04,25.69,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.97,54.0,H-SC,1.80,220338.0,-49329.0,85667.0,-0.54,-18.29,38.88,13.47,138.0,-0.58,1.58,11.44,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-20.69,50.0,H-MC,7.85,107378.0,-24457.0,66682.0,-0.59,-18.55,62.10,32.03,98.0,-0.37,0.77,20.27,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.12,54.0,H-LC,12.83,293985.0,-51971.0,126825.0,0.50,-15.02,43.14,21.64,1.0,-0.41,2.10,4.15,X40,ATH,IT
40,INFY,2275.00,-17.80,52.0,H-LC,8.82,320672.0,7784.0,163895.0,-0.19,2.49,51.11,54.87,3.0,0.05,2.29,9.30,X40,BTT,IT
42,ITC,452.00,-37.50,53.0,H-LC,2.87,201884.0,1746.0,19139.0,0.05,0.87,9.48,10.44,4.0,0.09,1.44,6.48,X40,NTT,FMCG
84,VBL,671.64,-17.63,35.0,H-LC,5.63,296383.0,-19459.0,132128.0,-1.51,-6.16,44.58,35.67,5.0,-0.15,2.12,6.84,X40N,ATH,FMCG
1,ABB,7934.00,-38.70,61.0,H-LC,11.14,256008.0,-5611.0,124830.0,-0.10,-2.14,48.76,45.57,7.0,-0.04,1.83,8.93,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,66.0,M-MC,6.97,229543.0,4581.0,163067.0,0.38,2.04,71.04,74.53,192.0,0.03,1.64,34.95,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,10.86,58.0,H-MC,3.68,191440.0,26770.0,41198.0,1.34,16.26,21.52,41.28,89.0,0.65,1.37,25.54,X40N,ATH,AC
85,VOLTAS,1918.49,1.58,60.0,H-MC,2.68,213060.0,21318.0,74720.0,0.32,11.12,35.07,50.08,99.0,0.29,1.52,18.67,XY25,ATH,AC
30,HAVELLS,2069.16,1.33,65.0,H-MC,1.82,251222.0,3491.0,71573.0,0.95,1.41,28.49,30.30,92.0,0.05,1.80,14.89,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.66,55.0,H-MC,6.55,209820.0,12792.0,76836.0,-0.54,6.49,36.62,45.49,102.0,0.17,1.50,20.38,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114756.0,14252.0,16387.0,-0.12,14.18,14.28,30.49,79.0,0.87,0.82,51.18,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.40,58.0,M-SC,0.63,102420.0,10922.0,67833.0,1.64,11.94,66.23,86.07,223.0,0.16,0.73,48.73,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.53,53.0,H-SC,12.21,137975.0,14639.0,119128.0,-0.22,11.87,86.34,108.45,119.0,0.12,0.99,36.88,AR,ATH,MISC
87,WIPRO,420.00,-11.07,54.0,M-LC,5.59,156426.0,5481.0,103976.0,0.41,3.63,66.47,72.51,53.0,0.05,1.12,9.87,XR,NTT,IT
36,IEX,219.00,-30.16,53.0,H-SC,17.34,199560.0,-394.0,97625.0,-0.10,-0.20,48.92,48.63,115.0,-0.00,1.43,11.33,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,56.0,L-SC,41.96,87610.0,-25939.0,65988.0,2.72,-22.84,75.32,35.27,268.0,-0.39,0.63,114.43,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.67,47.0,H-SC,13.33,88795.0,-12168.0,106696.0,0.17,-12.05,120.16,93.63,148.0,-0.11,0.64,32.26,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.09,60.0,H-SC,12.94,96465.0,-1515.0,23123.0,0.52,-1.55,23.97,22.05,152.0,-0.07,0.69,40.22,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114756.0,14252.0,16387.0,-0.12,14.18,14.28,30.49,79.0,0.87,0.82,51.18,MH,ATH,METALS
8,ATULAUTO,844.00,3905.41,65.0,M-SC,6.45,124809.0,-22559.0,74374.0,2.12,-15.31,59.59,35.16,236.0,-0.30,0.89,27.48,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3905.41,65.0,M-SC,6.45,124809.0,-22559.0,74374.0,2.12,-15.31,59.59,35.16,236.0,-0.30,0.89,27.48,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,56.0,L-SC,41.96,87610.0,-25939.0,65988.0,2.72,-22.84,75.32,35.27,268.0,-0.39,0.63,114.43,XR,NTT,HOTELS
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114756.0,14252.0,16387.0,-0.12,14.18,14.28,30.49,79.0,0.87,0.82,51.18,MH,ATH,METALS
61,SAMMAANCAP,326.00,-170.45,62.0,M-SC,2.63,81630.0,-20580.0,113972.0,0.18,-20.14,139.62,91.37,208.0,-0.18,0.58,32.51,XY25,NTT,FINANCE
27,GREENPANEL,537.00,235.27,65.0,M-SC,4.13,149792.0,-33286.0,112269.0,-0.45,-18.18,74.95,43.14,230.0,-0.30,1.07,40.90,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.45
1,25,42.70
2,50,73.09


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.25
LC    33.93
MC    22.76
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.48
X40      15.01
X40N     12.75
XR       11.40
XY25      9.61
AR        8.51
OX40N     8.05
X200      1.73
MH        1.69
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.36
H-LC    25.81
H-MC    19.84
M-SC    13.36
M-LC     7.07
M-MC     2.62
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.13,-14.45,76.85
FMCG,11.59,-4.39,40.92
FINANCE,9.50,-15.73,61.06
BANKS,7.63,-12.43,64.19
MISC,7.38,-11.46,73.46
PAINTS,5.72,-15.52,32.74
ELECTRICAL,5.60,-5.15,63.10
AC,3.59,3.57,29.09
INSURANCE,3.36,-7.25,44.03


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2863005.0,22
XR,1317653.0,14
AR,1305108.0,9
X40,749457.0,10
X40N,628797.0,10
OX40N,523396.0,10
XY25,406548.0,7
SR,245010.0,2
X5K,116537.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3447671.0,29
M-SC,1377614.0,17
H-LC,1263844.0,15
H-MC,1179579.0,15
M-LC,391292.0,5
M-MC,311699.0,2
L-SC,247633.0,3
L-MC,60361.0,1
L-LC,38111.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1042550.0      6
M-SC       XY24       796862.0      7
H-SC       AR         784637.0      5
           XR         764704.0      7
H-MC       XY24       488946.0      4
H-LC       X40        470800.0      5
           AR         357852.0      2
M-MC       XY24       311699.0      2
H-SC       X40N       298422.0      4
           OX40N      250256.0      4
           SR         245010.0      2
H-LC       X40N       229736.0      4
H-MC       X40        210021.0      4
           XY25       184992.0      2
L-SC       XR         163742.0      2
M-SC       AR         162619.0      2
M-LC       XY24       161750.0      2
M-SC       OX40N      122567.0      4
M-LC       X5K        116537.0      1
M-SC       XY25       113972.0      1
           XR         112958.0      2
H-MC       XR         111912.0      1
M-LC       XR         103976.0      1
H-MC       X40N       100639.0      2
L-SC       OX40N       83891.0      1
H-LC       X200        83814.0      1
M-SC       X40         68636.0      1
H-MC       OX40N       66682.0      1
H-SC       MH          62092.0      1
H-LC       XY24        61198.0      1
           XY25        60444.0      2
L-MC       XR          60361.0      1
L-LC       XY25        38111.0      1
H-MC       MH          16387.0      1
M-LC       XY25         9029.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
